# <font color=green>APR - Travaux Pratiques n°8.</font>

> **Ce sujet est en lien avec le quatrième chapitre du cours, et concerne la programmation CUDA. Les mêmes commentaires que ceux des derniers TP s’appliquent ici aussi.**
>
> **En imagerie numérique, l'égalisation d'histogramme est une méthode d'ajustement du contraste d'une image donnée (cf. http://en.wikipedia.org/wiki/Histogram_equalization). Pour une image en niveaux de gris, l'idée est de calculer un histogramme comptant l'utilisation de chaque niveau de gris, de calculer la fonction de répartition de cet histogramme, puis d'étaler les niveaux de gris utilisés.**
>
> **Plus précisément, soit $\left\{x_i\right\}$ l'ensemble des pixels d'une image définie sur $L$ niveaux de gris. L'histogramme est un tableau comptant les occurrences de chaque niveau de gris noté $l$, pour $l\in\left[0\ldots L-1\right]$ : $$h\left(l\right)=\sum_{i=0}^{n-1}\delta\left(x_i-l\right),$$ où $n$ est le nombre de pixels de l'image, et $\delta$ est la fonction de Dirac telle que : $$\delta\left(\xi\right)=\left\{\begin{matrix}1\mathrm{\ si\ }\xi=0,\\0\mathrm{\ sinon.}\\\end{matrix}\right. $$**
>
> **La fonction de répartition $r$ est définie sur l'intervalle des niveaux de gris comme la somme des nombres d'occurrence des valeurs précédentes :**
$$r\left(l\right)=\sum_{k=0}^{l}h\left(k\right).$$
>
> **Eh oui, c’est une somme préfixe, donc un SCAN inclusif ! La transformation suivante permet « d’étaler » l'histogramme :**
$$
T\left(x_i\right)=\frac{L-1}{n}r\left(x_i\right).
$$
>
> **Cette méthode est étendue aux images couleurs en appliquant cette transformation sur la composante « intensité » (V) de la couleur exprimée dans le repère HSV : Hue (Teinte), Saturation et Value (cf. http://en.wikipedia.org/wiki/HSL_and_HSV). Avec des images 24 bits, la valeur s’exprime sur 1 octet ; donc L vaut 256 (et L-1=255).**
>
> **Nous allons jouer avec l’implantation de l’histogramme et le scan inclusif.**
>
> **<font color=pink>N'oubliez d'exécuter les quatre premières cellules de code afin d'installer l'extension CUDA et de vérifier son bon fonctionnement.</font>**

## <font color=green>Installation du sous-sytème</font>

In [ ]:
# Ne lancer que si vous avez des problèmes du genre "the provided PTX was compiled with an unsupported toolchain.."
!sudo apt update 2>&1 > /dev/null
!sudo apt remove --autoremove -y --allow-change-held-packages cuda-toolkit-config-common cuda-toolkit-12-config-common cuda-toolkit-12-*-config-common libcublas-12-* libcublas-dev-12-* 2>&1 > /dev/null
!sudo apt list | grep cuda-toolkit
!sudo apt install -y cuda-toolkit-12-0-config-common cuda-nvcc-12-0 2>&1 > /dev/null

In [ ]:
# vérifions l'installation du SDK Cuda ...
!/usr/local/cuda/bin/nvcc --version

In [ ]:
# Installons gdown pour charger fichier depuis Google Drive
!pip install --upgrade --no-cache-dir gdown &> /dev/null
# importation Python pour charger/afficher des images
from google.colab.patches import cv2_imshow
import cv2
import numpy as np


def afficher(file, width):
  img = cv2.imread(file)
  height = int(img.shape[0] * width / float(img.shape[1]))
  resized = cv2.resize(img, (width, height), interpolation = cv2.INTER_AREA)
  cv2_imshow(resized)


def afficher3(file, width, extension):
  img = cv2.imread(file+".ppm")
  height = int(img.shape[0] * width / float(img.shape[1]))
  resizedOrig = cv2.resize(img, (width, height), interpolation = cv2.INTER_AREA)
  img = cv2.imread(file+"_equalized_"+extension+"_reference.ppm")
  resizedRef = cv2.resize(img, (width, height), interpolation = cv2.INTER_AREA)
  img = cv2.imread(file+"_equalized_"+extension+".ppm")
  resizedResult = cv2.resize(img, (width, height), interpolation = cv2.INTER_AREA)
  cv2_imshow(np.hstack((resizedOrig, resizedRef, resizedResult)))

---
# <font color=green>TP</font>
> L'installation s'est bien déroulée ? Parfait, maintenant au travail !
>
> En premier, il faut charger le TP8 depuis le drive Google ... Vous pouvez charger ce fichier (*i.e.* le premier, le second contient des images) sur votre ordinateur pour étudiez les interfaces, bien que la plupart soient dans le cours ...


In [ ]:
# Chargeons le TP8
!gdown https://drive.google.com/uc?id=1LdDZmQEh0rH7kaQmDTsmpRbMafP8AXan
!unzip -oqq TP8.zip
!ls TP8/Images


>
> Le code du TP est dans le répertoire TP8. Vous pouvez le vérifier dans une cellule en tapant " !ls TP8" par exemple ...
>
> Nous démarrons avec l'exercice 1.
---
## <font color=green>Exercice 1</font>
>
> **Implémentez la fonction `rgb2hsv` qui, pour chaque pixel de l'image, calcule sa valeur dans l'espace HSV en utilisant la fonction `RGB2HSV`, et répartit le résultat dans trois tableaux différents. Notez qu'il s'agit d'une forme de SCATTER. Ce type de répartition en trois tableaux vise à optimiser le débit mémoire d'un kernel CUDA (encore la coalescence).**
>
> **Implémentez la transformation inverse (`hsv2rgb`), de HSV vers RGB, en utilisant la fonction `HSV2RGB`.**
**
>
> ### <font color=green>Partie étudiante</font>
>
> La partie ci-dessous est pour vous. Répondez à l'exercice dans la cellule suivante.
>
> Pour sauvegarder, n'oubliez pas de terminer par "Shift-Entrée" ...
>
> **<font color=pink>Attention : ne touchez pas à la première ligne !</font>**

In [ ]:
%%writefile TP8/student/exo1/student.cu
#include <iostream>
#include <exo1/student.h>
#include <OPP_cuda.cuh>
#include <float.h>

namespace
{
	__device__
	float3 RGB2HSV( const uchar3 inRGB ) {
		const float R = float( inRGB.x ) / 256.f;
		const float G = float( inRGB.y ) / 256.f;
		const float B = float( inRGB.z ) / 256.f;

		const float min		= fminf( R, fminf( G, B ) );
		const float max		= fmaxf( R, fmaxf( G, B ) );
		const float delta	= max - min;

		// H
		float H;
		if		( delta < FLT_EPSILON )
			H = 0.f;
		else if	( max == R )
			H = 60.f * ( G - B ) / ( delta + FLT_EPSILON ) + 360.f;
		else if ( max == G )
			H = 60.f * ( B - R ) / ( delta + FLT_EPSILON ) + 120.f;
		else
			H = 60.f * ( R - G ) / ( delta + FLT_EPSILON ) + 240.f;
		while	( H >= 360.f )
			H -= 360.f ;

		// S
		const float S = max < FLT_EPSILON ? 0.f : 1.f - min / max;

		// V
		const float V = max;

		return make_float3( H, S, V*256.f );
	}

	__device__
	uchar3 HSV2RGB( const float H, const float S, const float V ) {
		const float	d	= H / 60.f;
		const int	hi	= int(d) % 6;
		const float f	= d - float(hi);

		const float Vn = V / 256.f;
		const float l   = Vn * ( 1.f - S );
		const float m	= Vn * ( 1.f - f * S );
		const float n	= Vn * ( 1.f - ( 1.f - f ) * S );

		float R, G, B;

		if		( hi == 0 )
			{ R = Vn; G = n;	B = l; }
		else if ( hi == 1 )
			{ R = m; G = Vn;	B = l; }
		else if ( hi == 2 )
			{ R = l; G = Vn;	B = n; }
		else if ( hi == 3 )
			{ R = l; G = m;	B = Vn; }
		else if ( hi == 4 )
			{ R = n; G = l;	B = Vn; }
		else
			{ R = Vn; G = l;	B = m; }

		return make_uchar3( R * 256.f, G * 256.f, B * 256.f );
	}

	__global__
	void RGB2HSV_kernel(
		uchar3 const*const source,
		float *const Hue,
		float *const Saturation,
		float *const Value,
		const unsigned size
	) {
		const unsigned tid = threadIdx.x + blockIdx.x * blockDim.x;
		if( tid < size )
		{
			float3 hsv( RGB2HSV(source[tid]) );
			Hue[tid] = hsv.x;
			Saturation[tid] = hsv.y;
			Value[tid] = hsv.z;
		}
	}

	__global__
	void HSV2RGB_kernel(
		float const*const Hue,
		float const*const Saturation,
		float const*const Value,
		uchar3 *const result,
		const unsigned size
	) {
		const unsigned tid = threadIdx.x + blockIdx.x * blockDim.x;
		if( tid < size )
		{
			result[tid] = HSV2RGB(Hue[tid], Saturation[tid], Value[tid]);
		}
	}
}

void StudentWorkImpl::run_RGB2HSV(
	OPP::CUDA::DeviceBuffer<uchar3>& dev_source,
	OPP::CUDA::DeviceBuffer<float>& dev_Hue,
	OPP::CUDA::DeviceBuffer<float>& dev_Saturation,
	OPP::CUDA::DeviceBuffer<float>& dev_Value,
	const unsigned width,
	const unsigned height
) {
	const unsigned fullsize = width * height ;
	const unsigned threads(1024);
	const unsigned blocksize = (fullsize + threads - 1) / threads;

	RGB2HSV_kernel<<<blocksize,threads>>>(
		dev_source.getDevicePointer(),
		dev_Hue.getDevicePointer(),
		dev_Saturation.getDevicePointer(),
		dev_Value.getDevicePointer(),
		fullsize);

		cudaDeviceSynchronize();
}

void StudentWorkImpl::run_HSV2RGB(
	OPP::CUDA::DeviceBuffer<float>& dev_Hue,
	OPP::CUDA::DeviceBuffer<float>& dev_Saturation,
	OPP::CUDA::DeviceBuffer<float>& dev_Value,
	OPP::CUDA::DeviceBuffer<uchar3>& dev_result,
	const unsigned width,
	const unsigned height
) {
	// TODO: map
	const unsigned fullsize =  width * height;
	const unsigned threads(1024);
	const unsigned nbblock = (fullsize + threads - 1) / threads;

	HSV2RGB_kernel<<<blocksize,threads>>>(
		dev_Hue.getDevicePointer(),
		dev_Saturation.getDevicePointer(),
		dev_Value.getDevicePointer(),
		dev_result.getDevicePointer(),
		fullsize);

		cudaDeviceSynchronize();
}

> ### <font color=green>Compilation</font>
> Exécutez la cellule suivante pour compiler le code ...

In [ ]:
!cd TP8 && sh ./build.sh exo1

> ### <font color=green>Exécution</font>
> Exécutez la cellule suivante pour exécuter le code ...

In [ ]:
# launch student work
!./TP8/linux/exo1 -i=TP8/Images/Nuit.ppm
# display input image
print("\nInput image is:")
afficher(file="TP8/Images/Nuit.ppm", width=600)
# display result
print("\nYour result is:")
afficher(file="TP8/Images/Nuit_RGB2HSV2RGB.ppm", width = 600)

## <font color=green>Exercice 2</font>

> **Calculez l’histogramme des valeurs $h(l)$.**
>
> ### <font color=green>Partie étudiante</font>
>
> La partie ci-dessous est pour vous. Répondez à l'exercice dans la cellule suivante.
>
> Pour sauvegarder, n'oubliez pas de terminer par "Shift-Entrée" ...
>
> **<font color=pink>Attention : ne touchez pas à la première ligne !</font>**


In [ ]:
%%writefile TP8/student/exo2/student.cu
#include <iostream>
#include <exo2/student.h>
#include <OPP_cuda.cuh>

namespace
{
  struct convert{
    __device__
        unsigned operator()(const float& i) const {
            return static_cast<unsigned>(i);
        }
  };
}

void StudentWorkImpl::run_Histogram(
	OPP::CUDA::DeviceBuffer<float>& dev_value,
	OPP::CUDA::DeviceBuffer<unsigned>& dev_histogram,
	const unsigned width,
	const unsigned height
) {
  OPP::CUDA::computeHistogram<float, unsigned, convert>(dev_value, dev_histogram, convert());
  cudaDeviceSynchronize();
}

> ### <font color=green>Compilation</font>
> Exécutez la cellule suivante pour compiler le code ...

In [ ]:
#!cat TP8/utils/OPP/OPP_cuda_histogram.cuh
!cd TP8 ; sh ./build.sh exo2

> ### <font color=green>Exécution</font>
> Exécutez les cellules suivantes pour exécuter le code (avec les images pré-chargées) ...

In [ ]:
# launch student work
!cd ./TP8 && ./linux/exo2 -i=./Images/Nuit.ppm

In [ ]:
# launch student work
!cd TP8 && linux/exo2 -i=./Images/Roy_Lichtenstein_Drowning_Girl.ppm

In [ ]:
# launch student work
!cd TP8 && linux/exo2 -i=./Images/The_Nightwatch_by_Rembrandt.ppm

## <font color=green>Exercice 3</font>

> **Calculez la fonction de répartition $r(l)$.**
>
> ### <font color=green>Partie étudiante</font>
>
> La partie ci-dessous est pour vous. Répondez à l'exercice dans la cellule suivante.
>
> Pour sauvegarder, n'oubliez pas de terminer par "Shift-Entrée" ...
>
> **<font color=pink>Attention : ne touchez pas à la première ligne !</font>**


In [ ]:
%%writefile TP8/student/exo3/student.cu
#include <iostream>
#include <exo3/student.h>
#include <OPP_cuda.cuh>

namespace
{
}

void StudentWorkImpl::run_Repartition(
	OPP::CUDA::DeviceBuffer<unsigned>& dev_histogram,
	OPP::CUDA::DeviceBuffer<unsigned>& dev_repartition
) {
	// TODO
	OPP::CUDA::inclusiveScan<unsigned,OPP::CUDA::Plus<unsigned>>(dev_histogram,dev_repartition,OPP::CUDA::Plus<unsigned>());
}

> ### <font color=green>Compilation</font>
> Exécutez la cellule suivante pour compiler le code ...

In [ ]:
!cd TP8 ; sh ./build.sh exo3

> ### <font color=green>Exécution</font>
> Exécutez les cellules suivantes pour exécuter le code (avec les images pré-chargées) ...

In [ ]:
# launch student work
!cd TP8 && linux/exo3 -i=./Images/Nuit.ppm

In [ ]:
# launch student work
!cd TP8 && linux/exo3 -i=./Images/Roy_Lichtenstein_Drowning_Girl.ppm

In [ ]:
# launch student work
!cd TP8 && linux/exo3 -i=./Images/The_Nightwatch_by_Rembrandt.ppm

## <font color=green>Exercice 4</font>

> **Calculez la transformation finale $T(x_i)$.**
>
> **Admirez.**
>
> **Comme toujours, votre rapport doit discuter les performances en fonctions des nombres de threads et de leur répartition.**
>
> ### <font color=green>Partie étudiante</font>
>
> La partie ci-dessous est pour vous. Répondez à l'exercice dans la cellule suivante.
>
> Pour sauvegarder, n'oubliez pas de terminer par "Shift-Entrée" ...
>
> **<font color=pink>Attention : ne touchez pas à la première ligne !</font>**

In [ ]:
%%writefile TP8/student/exo4/student.cu
#include <iostream>
#include <exo4/student.h>
#include <OPP_cuda.cuh>

namespace
{
	__global__
		void transform(const float* const v , const unsigned* const r,float* const t , const unsigned size){
			const unsigned tid = blockIdx.x * blockDim.x + threadIdx.x;

			if(tid < size){
				const unsigned char convert =static_cast<unsigned char>(v[tid]);

				t[tid] = (255.f * static_cast<float>(r[convert])) / static_cast<float>(size);
			}
		}
}

void StudentWorkImpl::run_Transformation(
	OPP::CUDA::DeviceBuffer<float>& dev_Value,
	OPP::CUDA::DeviceBuffer<unsigned>& dev_repartition,
	OPP::CUDA::DeviceBuffer<float>& dev_transformation // or "transformed"
) {
	// TODO
	const unsigned nbthreads(1024);
	const unsigned size = dev_Value.getNbElements();

	const dim3 threads(nbthreads);
	const dim3 nbBlocs((size + nbthreads -1) / nbthreads);

	transform<<<nbBlocs,threads>>>(
			dev_Value.getDevicePointer(),
			dev_repartition.getDevicePointer(),
			dev_transformation.getDevicePointer(),
			size

	);

}

bool StudentWorkImpl::isImplemented() const { return true; }

> ### <font color=green>Compilation</font>
> Exécutez la cellule suivante pour compiler le code ...

In [ ]:
!cd TP8 ; sh ./build.sh exo4

> ### <font color=green>Exécution</font>
> Exécutez les cellules suivantes pour exécuter le code (avec les images pré-chargées) ...

In [ ]:
# launch student work
!cd TP8 && linux/exo4 -i=./Images/Hopper.railroad.ppm
# width = mettez une largeur en fonction de votre bande passante Internet
afficher3(file="TP8/Images/Hopper.railroad", width=400, extension="HE")

In [ ]:
# launch student work
!cd TP8 && linux/exo4 -i=./Images/Nuit.ppm
# width = mettez une largeur en fonction de votre bande passante Internet
afficher3(file="TP8/Images/Nuit", width=400, extension="HE")

In [ ]:
# launch student work
!cd TP8 && linux/exo4 -i=./Images/Paris.ppm
# width = mettez une largeur en fonction de votre bande passante Internet
afficher3(file="TP8/Images/Paris", width=400, extension="HE")

In [ ]:
# launch student work
!cd TP8 && linux/exo4 -i=./Images/Roy_Lichtenstein_Drowning_Girl.ppm
# width = mettez une largeur en fonction de votre bande passante Internet
afficher3(file="TP8/Images/Roy_Lichtenstein_Drowning_Girl", width=400, extension="HE")

In [ ]:
# launch student work
!cd TP8 && linux/exo4 -i=./Images/SunFlowers.ppm
# width = mettez une largeur en fonction de votre bande passante Internet
afficher3(file="TP8/Images/SunFlowers", width=400, extension="HE")

In [ ]:
# launch student work
!cd TP8 && linux/exo4 -i=./Images/The_Nightwatch_by_Rembrandt.ppm
# width = mettez une largeur en fonction de votre bande passante Internet
afficher3(file="TP8/Images/The_Nightwatch_by_Rembrandt", width=400, extension="HE")

In [ ]:
# launch student work
!./TP8/linux/exo4 -i=TP8/Images/Unequalized_Hawkes_Bay_NZ.ppm
# width = mettez une largeur en fonction de votre bande passante Internet
afficher3(file="TP8/Images/Unequalized_Hawkes_Bay_NZ", width=400, extension="HE")

## <font color=green>Exercice 5</font>

> **Implantez la version AHE décrite dans la section III.A, équation 6, de l’article suivant :**
>
> <a href="https://perso.telecom-paristech.fr/angelini/SI241/papers_for_project/test.pdf">T. Arici, S. Dikbas, and Y. Altunbasak, A histogram modification framework and its application for image contrast enhancement, IEEE Trans. Image Process., vol. 18, no. 9, pp. 1921–1935, Sep. 2009.</a>
>
> **Expliquez les choix ayant conduit à votre implantation de cette méthode.**
>
> **Notez ici que la transformation de l’histogramme, si elle était directement implantée avant le calcul de la fonction de répartition en utilisant des entiers, donnerait des erreurs numériques importantes. Pour éviter cela, il vous est demandé d’appliquer l’équation (6) à la volée lors de la transformation finale de la valeur V de chaque pixel. Cela est rendu possible, car l’équation (6) n’est ni plus ni moins qu’une application linéaire (et donc s’applique aussi bien sur les deux fonctions de répartition résultant des deux histogrammes $h$ et $u$).**
>
> **Notez enfin que pour $\lambda=0$ vous retrouvez l’algorithme HE…**


>
> ### <font color=green>Partie étudiante</font>
>
> La partie ci-dessous est pour vous. Répondez à l'exercice dans la cellule suivante.
>
> Pour sauvegarder, n'oubliez pas de terminer par "Shift-Entrée" ...
>
> **<font color=pink>Attention : ne touchez pas à la première ligne !</font>**

In [ ]:
%%writefile TP8/student/exo5/student.cu
#include <iostream>
#include <exo5/student.h>
#include <OPP_cuda.cuh>

namespace
{
		__global__
		void transform(const float lambda,const float* const v , const unsigned* const r,float* const t , const unsigned size){
			const unsigned tid = blockIdx.x * blockDim.x + threadIdx.x;

			if(tid < size){
				const unsigned char convert =static_cast<unsigned char>(v[tid]);

			const float lambdaComplement = 1 / (1 + lambda);
      const float histogramValue = static_cast<float>(r[convert]);
      const float lambdaRatio = lambda / (1 + lambda);
      const float sizeRatio = static_cast<float>(size) * static_cast<float>(convert) / 256;

				t[tid] = (255.f * ( (lambdaComplement * histogramValue) + (lambdaRatio * sizeRatio) )) / static_cast<float>(size);
			}


		}
}

void StudentWorkImpl::run_Transformation(
	const float lambda, // value to use in equation (6)
	OPP::CUDA::DeviceBuffer<float>& dev_Value,
	OPP::CUDA::DeviceBuffer<unsigned>& dev_repartition,
	OPP::CUDA::DeviceBuffer<float>& dev_transformation // or "transformed"
) {
	// TODO: a map
	// NB: equation (6) is applied on the fly to the transformed value...

	const unsigned nbthreads(1024);
	const unsigned size = dev_Value.getNbElements();

	const dim3 threads(nbthreads);
	const dim3 nbBlocs((size + nbthreads -1) / nbthreads);

	transform<<<nbBlocs,threads>>>(
			lambda,
			dev_Value.getDevicePointer(),
			dev_repartition.getDevicePointer(),
			dev_transformation.getDevicePointer(),
			size

	);
}

> ### <font color=green>Compilation</font>
> Exécutez la cellule suivante pour compiler le code ...

In [ ]:
!cd TP8 ; sh ./build.sh exo5

> ### <font color=green>Exécution</font>
> Exécutez les cellules suivantes pour exécuter le code (avec les images pré-chargées) ...

In [ ]:
# launch student work
!./TP8/linux/exo5 -i=TP8/Images/Hopper.railroad.ppm -l=1
# width = mettez une largeur en fonction de votre bande passante Internet
afficher3(file="TP8/Images/Hopper.railroad", width=400, extension="AHE")

In [ ]:
# launch student work
!./TP8/linux/exo5 -i=TP8/Images/Nuit.ppm -l=1
# width = mettez une largeur en fonction de votre bande passante Internet
afficher3(file="TP8/Images/Nuit", width=400, extension="AHE")

In [ ]:
# launch student work
!./TP8/linux/exo5 -i=TP8/Images/Paris.ppm -l=1
# width = mettez une largeur en fonction de votre bande passante Internet
afficher3(file="TP8/Images/Paris", width=400, extension="AHE")

In [ ]:
# launch student work
!./TP8/linux/exo5 -i=TP8/Images/Roy_Lichtenstein_Drowning_Girl.ppm -l=1
# width = mettez une largeur en fonction de votre bande passante Internet
afficher3(file="TP8/Images/Roy_Lichtenstein_Drowning_Girl", width=400, extension="AHE")

In [ ]:
# launch student work
!./TP8/linux/exo5 -i=TP8/Images/SunFlowers.ppm -l=1
# width = mettez une largeur en fonction de votre bande passante Internet
afficher3(file="TP8/Images/SunFlowers", width=400, extension="AHE")

In [ ]:
# launch student work
!./TP8/linux/exo5 -i=TP8/Images/The_Nightwatch_by_Rembrandt.ppm -l=1
# width = mettez une largeur en fonction de votre bande passante Internet
afficher3(file="TP8/Images/The_Nightwatch_by_Rembrandt", width=400, extension="AHE")

In [ ]:
# launch student work
!./TP8/linux/exo5 -i=TP8/Images/Unequalized_Hawkes_Bay_NZ.ppm -l=1
# width = mettez une largeur en fonction de votre bande passante Internet
afficher3(file="TP8/Images/Unequalized_Hawkes_Bay_NZ", width=400, extension="AHE")

## <font color=green>Exercice 6</font>
>
> **Implantez la version WHE décrite section III.C de l’article vu dans l’exercice précédent (équation 12). Notez ici que dans l’article la phrase « the average local variance of pixels with gray-level i », doit être lue pour des images en couleur en remplaçant « niveau de gris » par « valeur ».**
>
> **Une erreur classique sur le calcul de la variance est de considérer comme correcte la formule suivante :**
>
> $$\sigma^2=\frac{1}{n}\sum\left(x_i-\bar{X}\right)^2$$
>
> **où $\bar{X}$ est la moyenne empirique :**
>
> $$\bar{X}=\frac{1}{n}\sum x_i.$$
>
> **Cette formulation de la variance est correcte en théorie, mais en utilisant des données réelles (comme le voisinage d’un pixel) alors cette formule est celle dite de la variance empirique. Or, cette variance empirique est biaisée… Pour corriger ce défaut, il faut utiliser la formulation suivante :**
>
> $$\widetilde{\sigma^2}=\frac{n}{n-1}\sigma^2=\frac{1}{n-1}\sum\left(x_i-\bar{X}\right)^2.$$
>
> **Une petite subtilité concerne encore le calcul de la variance, celle locale aux pixels de chaque valeur $V$ dans l’algorithme AHE. Sachant que ces variances seront ensuite moyennées pour chaque valeur $V$, une possibilité est de la calculer localement avec deux pixels seulement : le pixel $X$ en position $\left(row,col\right)$ de valeur $V$, et le pixel $Y$ en position $\left(row,col+2\right)$ par exemple et de valeur $V_y$. Ce procédé évite un calcul coûteux dans une fenêtre autour de $X$ qui serait d’une taille à définir. Cette solution explique pourquoi la fonction à coder ne reçoit pas de taille de fenêtre de calcul de la variance locale.**
>
> **Dans le rapport, expliquez le calcul de la variance locale que vous effectuez.**
>
> **Notez ici que la fonction à coder doit réaliser tout le traitement à appliquer sur la valeur $V$ de chaque pixel : d’abord le calcul de l’histogramme et de la moyenne des variances locales, puis la transformation de l’histogramme, le calcul de la fonction de répartition et enfin l’application de la transformation…**

> ### <font color=green>Partie étudiante</font>
>
> La partie ci-dessous est pour vous. Répondez à l'exercice dans la cellule suivante.
>
> Pour sauvegarder, n'oubliez pas de terminer par "Shift-Entrée" ...
>
> **<font color=pink>Attention : ne touchez pas à la première ligne !</font>**

In [ ]:
%%writefile TP8/student/exo6/student.cu
#include <exo6/student.h>

#include <OPP_cuda.cuh>
#include <algorithm>
#include <cuda.h>
#include <iostream>
#include <type_traits>

namespace {

  /**
   * @brief print a device buffer on standard output
   *
   * @tparam T Datatype of the data stored into the DeviceDuffer
   * @param msg Message to display first (before the data)
   * @param deviceBuffer Contains the data to display
   */
  template <typename T>
  void print(std::string &msg, OPP::CUDA::DeviceBuffer<T> &deviceBuffer)
  {
    const auto size = deviceBuffer.getNbElements();
    std::vector<T> hostVector(size);
    deviceBuffer.copyToHost(hostVector.data());
    std::cout << "======= " << msg << " of size " << size << " =====\n";
    for (unsigned i{0u}; i < size; ++i) {
      std::cout << hostVector[i] << " ";
      if ((i % 16u) == 15u) {
        std::cout << "\n";
      }
    }
  }

  void buildHistogramAndVarianceSum(OPP::CUDA::DeviceBuffer<float> &dev_inputValue,
                                    OPP::CUDA::DeviceBuffer<unsigned> &dev_cdf,
                                    OPP::CUDA::DeviceBuffer<float> &dev_weight,
                                    const unsigned imageWidth)
  {
    // TODO
  }

  void buildCumulativeDistributionFunction(OPP::CUDA::DeviceBuffer<unsigned> &dev_cdf,
                                           OPP::CUDA::DeviceBuffer<float> &dev_weight,
                                           const float lambda,
                                           const unsigned size)
  {
    // TODO
  }

  void applyTransformation(OPP::CUDA::DeviceBuffer<float> &dev_inputValue,
                           OPP::CUDA::DeviceBuffer<unsigned> &dev_cdf,
                           OPP::CUDA::DeviceBuffer<float> &dev_outputValue)
  {
    // TODO
  }
} // namespace

void StudentWorkImpl::run_WHE(OPP::CUDA::DeviceBuffer<float> &dev_inputValue,
                              OPP::CUDA::DeviceBuffer<unsigned> &dev_histo,
                              OPP::CUDA::DeviceBuffer<float> &dev_weight,
                              OPP::CUDA::DeviceBuffer<float> &dev_outputValue,
                              const unsigned imageWidth,
                              const unsigned imageHeight,
                              const float lambda)
{
  // 1. calcul par valeur dans [0..255] de l'histogramme ET de la somme des variances/valeur
  ::buildHistogramAndVarianceSum(dev_inputValue, dev_histo, dev_weight, imageWidth);

  // ::print(std::string("histo"), dev_histo); // for debug, if needed
  // ::print(std::string("weight"), dev_weight); // for debug, if needed

  // 2. calcul de la CDF (dans histo pour économiser de la mémoire)
  ::buildCumulativeDistributionFunction(dev_histo, dev_weight, lambda, imageWidth * imageHeight);

  // 3. application de la transformation...
  ::applyTransformation(dev_inputValue, dev_histo, dev_outputValue);
}

> ### <font color=green>Compilation</font>
> Exécutez la cellule suivante pour compiler le code ...

In [ ]:
!cd TP8 ; sh ./build.sh exo6

> ### <font color=green>Exécution</font>
> Exécutez les cellules suivantes pour exécuter le code (avec les images pré-chargées) ...

In [ ]:
# launch student work
!./TP8/linux/exo6 -i=TP8/Images/Hopper.railroad.ppm -l=1
# width = mettez une largeur en fonction de votre bande passante Internet
afficher3(file="TP8/Images/Hopper.railroad", width=400, extension="WHE")

In [ ]:
# launch student work
!./TP8/linux/exo6 -i=TP8/Images/Nuit.ppm -l=1
# width = mettez une largeur en fonction de votre bande passante Internet
afficher3(file="TP8/Images/Nuit", width=400, extension="WHE")

In [ ]:
# launch student work
!./TP8/linux/exo6 -i=TP8/Images/Paris.ppm -l=1
# width = mettez une largeur en fonction de votre bande passante Internet
afficher3(file="TP8/Images/Paris", width=400, extension="WHE")

In [ ]:
# launch student work
!./TP8/linux/exo6 -i=TP8/Images/Roy_Lichtenstein_Drowning_Girl.ppm -l=1
# width = mettez une largeur en fonction de votre bande passante Internet
afficher3(file="TP8/Images/Roy_Lichtenstein_Drowning_Girl", width=400, extension="WHE")

In [ ]:
# launch student work
!./TP8/linux/exo6 -i=TP8/Images/SunFlowers.ppm -l=1
# width = mettez une largeur en fonction de votre bande passante Internet
afficher3(file="TP8/Images/SunFlowers", width=400, extension="WHE")

In [ ]:
# launch student work
!./TP8/linux/exo6 -i=TP8/Images/The_Nightwatch_by_Rembrandt.ppm -l=10
# width = mettez une largeur en fonction de votre bande passante Internet
afficher3(file="TP8/Images/The_Nightwatch_by_Rembrandt", width=400, extension="WHE")

In [ ]:
# launch student work
!./TP8/linux/exo6 -i=TP8/Images/Unequalized_Hawkes_Bay_NZ.ppm -l=1
# width = mettez une largeur en fonction de votre bande passante Internet
afficher3(file="TP8/Images/Unequalized_Hawkes_Bay_NZ", width=400, extension="WHE")

In [ ]:
from google.colab import files
u = files.upload()
print(u)

In [ ]:
files.download('TP8')

# <font color=green>That's all, folks!</font>